## TO DO
- [ ] <strike>Treinar um modelo, gerar novos dados apenas com volume e vazão diferentes, ver o desempenho</strike>
- [ ] <strike>Diferentes situações:</strike>
    - [x] reação inversa
    - [x] adiabática
- [x] <strike> Ver quanto tempo demora pra atingir o estado estacionário (provavelmente algumas amostras só)</strike>
- [x] <strike> Quantos pontos mínimos para treinar bem ?</strike>
- [ ] Gerar novo dataset com ordem 0, ver se outra função de ativação se adeua melhor

<H1>0. Import packages </H1>

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History
history = History()

<h2>1. Read data </h2>

In [ ]:
data_path = '../data_section/datas/data_n1000_order2.csv'
# data_path = '../data_section/datas/data_n1000_order2_adiabatic.csv'
# data_path = '../data_section/datas/data_n1000_order0.csv'
# data_path = '../data_section/datas/data_n100_order2.csv'
# data_path = '../data_section/datas/data_n50_order2.csv'
data = pd.read_csv(data_path, index_col=0)

outsample_data = pd.read_csv('../data_section/datas/outsample_test.csv', index_col=0)
V = 500

<h2>2. Select features</h2>

- Normalize it
- Split it in train and test sets

In [ ]:
x_cols = ['Cae', 'Cbe', 'Te']
y_col = ['Cc']

minmax = MinMaxScaler()
in_data = minmax.fit_transform(data[x_cols].to_numpy())

X_train, X_test, Y_train, Y_test = train_test_split(in_data, data[y_col].to_numpy(), test_size=.3, )
Y_test = Y_test.reshape(Y_test.shape[0])
Y_train = Y_train.reshape(Y_train.shape[0])

print(f'x train: {X_train.shape}, x test: {X_test.shape}')
print(f'y train: {Y_train.shape}, y test: {Y_test.shape}')

<h2>3. Create the network </h2>

- How many layers
- How many neurons per layer
- Which activation function
- Which metrics
- How long will be the training (using epochs)
- Whether use validation sets or not

In [ ]:
model = Sequential()
model.add(Dense(81, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(37, activation='exponential', ))
model.add(Dense(15, activation='sigmoid', ))
model.add(Dense(3, activation='exponential'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
hist = model.fit(X_train, Y_train, epochs=200, verbose=0, validation_split=.3, use_multiprocessing=True,
                 initial_epoch=50)
evaluate(model, X_train, X_test, Y_train, Y_test)

<h2>4. Validate the results</h2>

In [ ]:
plot_loss_history(hist)

In [ ]:
plot_identity_keras(model, X_test, Y_test)

In [ ]:
plot_rnn_keras(model)

In [ ]:
plot_first_layer_keras(model, label=x_cols)

In [ ]:
plot_boxplot_input(in_data, x_cols)

<H2>5. Save the model</H2>

In [ ]:
model.save('models/' + data_path.split('/')[-1].replace('data', 'model').replace('csv', 'h5'))

<h2>6. (OPTIONAL) Load the model</h2>

In [ ]:
model = load_model('models/model_n100_order2.h5')

In [ ]:
plot_identity_keras(model, in_data, data['Cc'])

In [ ]:
_outsample_data = outsample_data.copy()
for col in x_cols:
    _outsample_data = _outsample_data[(_outsample_data[col] > data[col].min()) & (_outsample_data[col] < data[col].max())]
_outsample_data_y = _outsample_data['Cc']
_outsample_data_x = minmax.fit_transform(_outsample_data[x_cols].to_numpy())

In [ ]:
plot_identity_keras(model, _outsample_data_x, _outsample_data_y)